# SIOPS 2022
> Neste notebook acessamos o catálogo de dados (datawarehouse) que cria o dataset do SIOPS 2022 e mostramos como ele é feito desde o acesso ao Banco de Dados da Aplicação SIOPS até a geração de arquivos Parquet com o dataset.
>
> Aproveitamos também para dar análises possíveis com esses arquivos usando Pandas.

In [1]:
# import json
import duckdb
from fastduck import database
import ipywidgets
# from fastprogress.fastprogress import master_bar, progress_bar
import pandas as pd
# import requests
# import os

Acessando o datawarehouse na pasta /data

In [ ]:
db = duckdb.connect('../data/catalog.db') 

In [ ]:
tables = (db.tables).df()
tables[tables['schema'].isin(['siops'])]

## Construção

### Tabelas da Aplicação SIOPS Usadas

In [ ]:
db.sql("select distinct table_name from duckdb_columns where schema_name = 'raw' and table_name like '%siops%'").df()

### TB_VL_VALORES

In [ ]:
db.sql("""SELECT COUNT(*) FROM RAW.SIOPS__TB_VL_VALORES""")

In [ ]:
db.sql("""SELECT COUNT(*) FROM RAW.SIOPSUF__TB_VL_VALORES""")

In [ ]:
21_879_872 + 173_590

### Unindo os dois bancos

In [ ]:
todos_valores = db.sql("""
    WITH
    -- TODOS_VALORES une tb_vl_valores de estados e municípios
    TODOS_VALORES AS (
        SELECT
            CO_MUNICIPIO AS CODIGO_IBGE,
            * EXCLUDE (CO_MUNICIPIO)
        FROM
            RAW.SIOPS__TB_VL_VALORES
        UNION ALL
        SELECT
            CO_UF AS CODIGO_IBGE,
            * EXCLUDE (CO_UF)
        FROM
            RAW.SIOPSUF__TB_VL_VALORES
    ) SELECT * FROM TODOS_VALORES
                       """)
todos_valores.df().head(5)

### Removendo as contas não operacionais (agregadoras)

A ideia do nosso dataset lancamentos é conter apenas dados inseridos pelos entes e remover dados derivados ainda que estejam salvos nos bancos

In [ ]:
db.sql(""" 
            SELECT * FROM SIOPS.CONTAS
        """)

In [ ]:
operacionais = db.sql(""" 
       SELECT * FROM todos_valores WHERE CO_ITEM in 
        (
            SELECT CODIGO_CONTA_SIOPS::TEXT FROM SIOPS.CONTAS WHERE TIPO_CONTA = 'operacional'
        )""")
operacionais

### Filtrando apenas dados de Receitas e Despesas
O banco do SIOPS é organizado de acordo como a interface gráfica funciona.  Portanto, precisamos dizer de que relatórios (PASTAS) e de que colunas (FASES entre outros conceitos) queremos valores.


In [ ]:
db.sql("""SELECT * FROM RAW.SIOPSUF__TB_PROJ_COLUNA""")

In [ ]:
receitas_despesas = db.sql(""" SELECT * FROM operacionais 
            -- 3_1 3_2... 3_18, 4_1..4_18..., 95_1..95_18 são as pastas_hierarquias válidas
            WHERE (REGEXP_MATCHES(co_pasta_hierarquia, '^(3|4|6|7|8|9|10|86|87|88|89|90|94|95)_([1-9]|1[0-8])$') OR CO_PASTA = 1) 
            AND CO_TIPO < 23 -- Remove fases que são Totais Gerais
            AND CO_TIPO NOT IN (14, 20, 21) -- Remove Restos a Pagar e Totalizadoras de Receitas para base de cálculo ASPS
            """)
receitas_despesas

### Filtrando dados homologados

Como o banco está reescrevendo por cima de valores já inseridos, checamos se a última atualização (cujos dados estão no banco) foram homologados e não incluímos no dataset caso não estejam.

In [ ]:
homologados = db.sql("""
                     SELECT
            P.PERIODO AS COMPETENCIA,
            V.*
        FROM
            receitas_despesas V
            JOIN STAGING.SIOPS__PERIODOS P ON P.ANO = V.NU_ANO
            AND P.CODIGO_BIMESTRE_SIOPS = V.NU_PERIODO
            JOIN STAGING.SIOPS__HOMOLOGADOS H ON H.PERIODO = P.PERIODO
            AND H.IBGE_ENTE = V.CODIGO_IBGE
                     """)
homologados

### Enriquecendo os dados


In [ ]:
final = db.sql(""" 
                SELECT
     H.COMPETENCIA as competencia, -- mudar para periodo?  exercicio?
     H.CODIGO_IBGE as ibge,
     S.ENTE as ente,
     CASE WHEN S.CAPITAL = 1 THEN 'S' ELSE 'N' END AS capital,
     S.REGIAO as regiao,
     S.UF as uf,
     CASE
         WHEN S.ESFERA = 'D' THEN 'Distrital'
         WHEN S.ESFERA = 'M' THEN 'Municipal'
         WHEN S.ESFERA = 'E' THEN 'Estadual'
         WHEN S.ESFERA = 'U' THEN 'Federal'
     END AS esfera,
     S.POPULACAO as populacao,
     -- remove conteúdo entre parenteses e espaços em branco
     TRIM(REGEXP_REPLACE(REGEXP_REPLACE(C.NO_COLUNA, '\s*=.*$', ''), '\s*\([a-z]\)', '')) AS fase,
    -- C.NO_COLUNA as fase,
     FS.FONTE as fonte,
     FS.SUBFUNCAO AS destinacao,
     CT.CODIGO_CONTA AS conta,
     CT.DESCRICAO_CONTA AS descricao_conta,
     H.NU_VALOR AS valor_nominal
 FROM
     homologados H 
     JOIN SIOPS.CONTAS CT ON CT.CODIGO_CONTA_SIOPS = H.CO_ITEM
     JOIN RAW.SIOPSUF__TB_PROJ_COLUNA C ON H.CO_TIPO = C.CO_SEQ_COLUNA
     LEFT JOIN STAGING.SIOPS__FONTES_SUBFUNCOES FS ON FS.PASTA_HIERARQUIA = H.CO_PASTA_HIERARQUIA
     JOIN SICONFI.ENTES S ON H.CODIGO_IBGE = S.codigo_ibge_6
 ORDER BY competencia, ibge, fase, conta, fonte, destinacao;
               """)
final

In [ ]:
dataset = db.sql("SELECT * FROM siops.lancamentos").df()
dataset

In [ ]:
!sqlmesh audit